# Entrenamiento iterativo

In [1]:
import os
import re
import json
import shutil
import joblib
import rasterio
import numpy as np
import pandas as pd
from glob import glob
from pyproj import CRS
from copy import deepcopy
from sqlite3 import connect
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from utilities import *

## Conjunto de entrenamiento

In [2]:
# cambiar según corresponda
# train_sqlite_files debe contener los .sqlite generados a partir de la verdad de campo
train_sqlite_files = glob('../data/verdad_campo_sqlite/*.sqlite')

train_data = pd.DataFrame()

for sf in train_sqlite_files:
    file_name = os.path.basename(sf)
    tile = re.search(r'\d+',file_name).group()
    cnx = connect(sf)
    df = pd.read_sql_query("SELECT * FROM output", cnx)
    df['tile_file'] = tile
    train_data = pd.concat([train_data, df], ignore_index=True)

In [3]:
train_data.shape

(466, 35)

In [4]:
train_data.head()

,ogc_fid,GEOMETRY,in1,id,cultivo,originfid,band_0,band_1,band_2,band_3,...,band_19,band_20,band_21,band_22,band_23,band_24,band_25,band_26,band_27,tile_file
0,1,b'\x01\x01\x00\x00\x00(I\x80\xe0\xf6fO\xc0\xbe...,014084,1,SOJA,7,0.42816,0.692352,0.36905,0.730617,...,4.816959,0.82200,0.410423,0.831812,0.543103,0.868833,2.95450,0.109526,0.861515,12544
1,2,"b'\x01\x01\x00\x00\x00\x1f|\x14""lkO\xc0^\x14\x...",014084,1,SOJA,8,0.41152,0.671952,0.42940,0.711504,...,4.774915,1.36300,0.441640,0.817090,0.587002,0.862946,3.15975,0.148569,0.859830,12544
2,3,b'\x01\x01\x00\x00\x00L).\x93\x08eO\xc0fz\xd9|...,014084,1,SOJA,11,0.49848,0.794412,0.42270,0.805747,...,5.247081,1.27025,0.423015,0.849940,0.563715,0.884401,3.21300,0.130448,0.879770,12544
3,4,b'\x01\x01\x00\x00\x00\x03X\x12\xb7\xb0iO\xc0[...,014084,1,SOJA,12,0.52872,0.834804,0.79800,0.835724,...,5.583365,2.86100,0.432937,0.861866,0.588915,0.892530,2.75850,0.430294,0.891874,12544
4,5,b'\x01\x01\x00\x00\x00\xa5\x87\x83F\xd6\\O\xc0...,014084,1,SOJA,28,0.50560,0.799404,0.32025,0.812116,...,5.668369,0.07450,0.453263,0.858904,0.589450,0.899895,2.88900,0.079681,0.894360,12544


## Conjunto de predicción

# cambiar según corresponda
# pred_sqlite_files debe contener los .sqlite generados a partir de la máscara mask_agri_aoi
pred_sqlite_files = glob('../data/selection_mask_agri_depto/*.sqlite')

pred_data = pd.DataFrame()

for sf in pred_sqlite_files:
    file_name = os.path.basename(sf)
    tile = re.search(r'\d+',file_name).group()
    cnx = connect(sf)
    df = pd.read_sql_query("SELECT * FROM output", cnx)
    df['tile_file'] = tile
    df.drop(columns=['id'], inplace=True)
    pred_data = pd.concat([pred_data, df], ignore_index=True)

pred_data.shape

pred_data.head()

## Agregado de departamentos

# train_data está dentro de pred_data
# saco esas filas de pred_data
# así no predecimos con lo mismo con lo que entrenamos

merged_data = pred_data.merge(train_data[['ogc_fid','tile_file','cultivo','id']], how='left', on=['ogc_fid','tile_file'], indicator=True)
train_data = merged_data[merged_data._merge=='both'].drop(columns=['_merge']).assign(id=lambda x:x.id.astype('int'))
pred_data = merged_data[merged_data._merge=='left_only'].drop(columns=['_merge'])

train_data.shape

pred_data.shape

## Entrenamiento del modelo

In [5]:
map_id2cultivo = dict((
    train_data[['id','cultivo']]
    .drop_duplicates()
    .assign(id=lambda x: x.id.astype('int'))
    .itertuples(index=False, name=None))
)
map_id2cultivo

{1: 'SOJA',
 2: 'MAIZ',
 3: 'MAIZ',
 5: 'GIRASOL',
 20: 'CAMPONATUR',
 10: 'ALFALFA',
 4: 'SOJA'}

In [6]:
le = LabelEncoder()
le.fit(train_data.id)

LabelEncoder()

In [7]:
map_le2id = dict(zip(le.transform(le.classes_), list(map(int,le.classes_))))

map_le2id

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 10, 6: 20}

In [8]:
train_data['id_le'] = le.transform(train_data.id)

train_data.head()

,ogc_fid,GEOMETRY,in1,id,cultivo,originfid,band_0,band_1,band_2,band_3,...,band_20,band_21,band_22,band_23,band_24,band_25,band_26,band_27,tile_file,id_le
0,1,b'\x01\x01\x00\x00\x00(I\x80\xe0\xf6fO\xc0\xbe...,014084,1,SOJA,7,0.42816,0.692352,0.36905,0.730617,...,0.82200,0.410423,0.831812,0.543103,0.868833,2.95450,0.109526,0.861515,12544,0
1,2,"b'\x01\x01\x00\x00\x00\x1f|\x14""lkO\xc0^\x14\x...",014084,1,SOJA,8,0.41152,0.671952,0.42940,0.711504,...,1.36300,0.441640,0.817090,0.587002,0.862946,3.15975,0.148569,0.859830,12544,0
2,3,b'\x01\x01\x00\x00\x00L).\x93\x08eO\xc0fz\xd9|...,014084,1,SOJA,11,0.49848,0.794412,0.42270,0.805747,...,1.27025,0.423015,0.849940,0.563715,0.884401,3.21300,0.130448,0.879770,12544,0
3,4,b'\x01\x01\x00\x00\x00\x03X\x12\xb7\xb0iO\xc0[...,014084,1,SOJA,12,0.52872,0.834804,0.79800,0.835724,...,2.86100,0.432937,0.861866,0.588915,0.892530,2.75850,0.430294,0.891874,12544,0
4,5,b'\x01\x01\x00\x00\x00\xa5\x87\x83F\xd6\\O\xc0...,014084,1,SOJA,28,0.50560,0.799404,0.32025,0.812116,...,0.07450,0.453263,0.858904,0.589450,0.899895,2.88900,0.079681,0.894360,12544,0


In [9]:
with open('../model/randomforest_parameters.json','r') as f:
    parameters = json.load(f)

In [10]:
def metadata_from_tile(in_raster):
    with rasterio.open(in_raster) as src:
        return(src.width, src.height, src.transform)

def sliding_windows(size, step_size, width, height, whole=False):
    """Slide a window of +size+ by moving it +step_size+ pixels"""
    w, h = size, size
    sw, sh = step_size, step_size
    end_i = height - h if whole else height
    end_j = width - w if whole else width
    for pos_i, i in enumerate(range(0, end_i, sh)):
        for pos_j, j in enumerate(range(0, end_j, sw)):
            real_w = w if whole else min(w, abs(width - j))
            real_h = h if whole else min(h, abs(height - i))
            yield Window(j, i, real_w, real_h), (pos_i, pos_j)

In [ ]:
next_train = pd.DataFrame()
i = 0
while True:
    
    # si el entrenamiento de la próxima iteración es mayor (en cantidad)
    # a la data de entrenamiento, entonces toma el entrenamiento de la próxima
    # iteración
    # si no, toma la data de entrenamiento original (verdad de capo original)
    # el entrenamiento de la próxima iteración es un df que se va enriqueciendo
    # con la nueva verdad de campo predicha
    # hago esto así porque, usando los tif, no sé cómo verificar si los pixeles
    # están o no en la verdad de campo (entonces no quiero agregar la nueva
    # verdad de camp al df que ya contiene la verdad de campo original
    # porquee estaría duplicando los datos)
    # así, primera iteración va a usar la data original y ya en la segunda
    # iteración va a tomar el entrenamiento con la nueva verdad
    if next_train.shape[0] >= train_data.shape[0]:
        train_data = next_train

    # arma carpeta para el output (i aumenta con las iteraciones)
    n_iter = '{0:03d}'.format(i)
    output_folder = os.path.join('..','model',f'randomforest_iterations_{n_iter}')
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.mkdir(output_folder)
    
    # segmenta en train y test
    X = train_data.filter(regex='band_').fillna(-99).to_numpy()
    y = train_data['id_le'].to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=.3, random_state=20220714, shuffle=True, stratify=y
    )
    
    # instancia y entrena el modelo con set de entrenamiento
    model = RandomForestClassifier(**parameters)
    model.fit(X_train, y_train)
    
    # predice sobre el conjunto de testeo
    probas = model.predict_proba(X_test)
    output_proba_file = os.path.join(output_folder, f'probas_{n_iter}.npy')
    np.save(output_proba_file, probas)
    
    y_hat = probas.argmax(axis=1)
    
    # guarda métricas
    cmatrix = confusion_matrix(y_test, y_hat, normalize='all')
    output_cmpatrix_file = os.path.join(output_folder, f'cmatrix_{n_iter}.npy')
    np.save(output_cmpatrix_file, cmatrix)
    
    report = classification_report(y_test, y_hat, output_dict=True)
    output_report_file = os.path.join(output_folder, f'report_{n_iter}.json')
    with open(output_report_file, 'w') as f:
        json.dump(report, f, ensure_ascii=False, indent=4)
    
    kappa = cohen_kappa_score(y_test, y_hat)
    output_kapp_file = os.path.join(output_folder, f'kapp_{n_iter}.txt')
    with open(output_kapp_file, 'w') as f:
        _ = f.write(str(kappa))
    
    # evalúa cantidad de aciertos y errores por nivel de confianza (cada 0.05)
    predictions = pd.DataFrame({
        'success':y_test==y_hat,
        'score':probas.max(axis=1),
    })
    hits, confidence = np.histogram(predictions[predictions.success==True].score, 20, (0,1))
    misses, _ = np.histogram(predictions[predictions.success==False].score, 20, (0,1))
    hits_misses_df = (
        pd
        .DataFrame({'hit':hits, 'miss':misses, 'confidence':np.round(confidence[:-1], 2)})
    )
    output_success_file = os.path.join(output_folder, f'hits_misses_{n_iter}.csv')
    hits_misses_df.to_csv(output_success_file, index=False)
    
    # selecciona el umbral
    # umbral = score cuya cantidad de aciertos duplique la cantidad de errores + 1
    threshold = hits_misses_df.loc[hits_misses_df.hit>(hits_misses_df.miss+1)*2,'confidence'].min()
    output_threshold_file = os.path.join(output_folder, f'threshold_{n_iter}.txt')
    with open(output_threshold_file, 'w') as f:
        _ = f.write(str(threshold))

    # instancia y entrena el modelo con set de entrenamiento + validación
    iter_X = train_data.filter(regex='band_').fillna(-99999).to_numpy()
    iter_y = train_data['id_le'].to_numpy()

    model = RandomForestClassifier(**parameters)
    model.fit(iter_X, iter_y)
    output_model_file = os.path.join(output_folder, f'model_{n_iter}.joblib')
    _ = joblib.dump(model, output_model_file)
    
    # levanta los .tif para predecir
    pred_tif = glob('../data/feature_importance/12*.tif')
    
    vc_len = X_train.shape[0] + X_test.shape[0]
    new_vc_len = 0
    for tif in pred_tif:
        # detecta nombre del raster
        # 12544.tif o 00000.tif
        name_raster = os.path.basename(tif)
        
        print(f'+++ PREDICCIÓN PARA TILE: {name_raster}')
        
        # levanta la metadata del tif
        width, height, transform = metadata_from_tile(tif)
        # arma las ventanas de 100x100
        windows = sliding_windows(100, 100, width, height)
    
        # si no está en la primera iteración
        # busca el raster para enmascarar de la iteración anterior
        if i>0:
            prev_i = i-1
            prev_iter = '{0:03d}'.format(prev_i)
            prev_folder = output_folder.replace(n_iter, prev_iter)
            prev_tif = os.path.join(prev_folder, name_raster)
        
        out_raster = os.path.join(output_folder, name_raster)
        with rasterio.open(
            out_raster, 'w', driver='GTiff', count=1,
            width=width, height=height, dtype=np.float64, transform=transform,
            crs=CRS.from_epsg(4326), compress='lzw') as dst:
            
            windows = list(windows)
            windows_len = len(windows)
            wn = 0
            for window in windows:
                print(f'... Prediciendo ventana {wn} de {windows_len}')
                print(window)
                wn +=1
                
                win=window[0]
                # abre ventana en el raster original
                src = rasterio.open(tif)
                img = src.read(window=win)
                r,m,n = img.shape
                
                # arma un dataframe con la data para predecir
                # shape: 10000 filas (100x100 pixeles) x 28 columnas (bandas)
                img_df = (
                    pd.DataFrame(img.reshape(r,m*n))
                    .T.fillna(-99)
                )
                img_df.rename(columns={col:f'band_{col}' for col in img_df.columns}, inplace=True)
                
                # si no está en la primera iteración
                # arma un dataframe con la data para enmascarar
                # shape: 10000 filas (100x100 pixeles) x 28 columnas (bandas)
                if i>0:
                    # abre ventana en el raster de la iteración previa
                    # para enmascarar
                    src_mask = rasterio.open(prev_tif)
                    mask = src_mask.read(window=win) 
                    mask_df = pd.DataFrame(mask.reshape(r,m*n)).T

                #img_df.rename(columns={col:f'band_{col}' for col in img_df.columns}, inplace=True)
                # agrega columna con predicciones
                iter_prediction = model.predict_proba(img_df.to_numpy())
                img_df['id_le'] = iter_prediction.argmax(axis=1)
                img_df['score'] = iter_prediction.max(axis=1)

                # si el score de la predicción supera al umbral
                # agrega esa info al entrenamiento de la próxima iteración
                new_vc = img_df[img_df.score>=threshold]
                new_vc['id_le'] = new_vc.id_le.astype('int')
                next_train = pd.concat([next_train, new_vc])
                new_vc_len += new_vc.shape[0]
                
                # asigna id real
                # enmascara las predicciones cuyo score no supera al umbral (-99)
                # y lo guarda en un nuevo .tif
                img_df['id'] = img_df.id_le.map(map_le2id)
                img_df.loc[img_df.score<threshold, 'id'] = -99
                print(img_df.id.unique())
                tif_class = np.expand_dims(img_df.id.to_numpy().reshape(n,m), axis=0)
                dst.write(tif_class, window=win)
                
                # cuando se hace la predicción hay que revisar si hay un .tif del modelo para enmascarar
                # (solo se predicen los pixeles que tengan NA)          
    
    output_vc_file = os.path.join(output_folder, f'verdad_campo_{n_iter}.txt')
    with open(output_vc_file, 'w') as f:
        _ = f.write(f'''verdad_campo_entrenamiento,{vc_len}\nvedad_campo_nueva{new_vc_len}\n''')
    
    
    # imprime información
    print(f'''\n*** ITERACIÓN #{i:03d}
    - Pixeles de entrenamiento: {X_train.shape[0]}
    - Pixeles de validación: {X_test.shape[0]}
    - Probabilidades sobre train guardadas en {output_proba_file}
    - Umbral definido: {threshold}
    - Pixeles del modelo final: {vc_len}
    - Modelo guardado en: {output_model_file}
    - Nueva verdad de campo predicha: {new_vc_len}''')
    
    if new_vc_len == 0:
        break

In [102]:
tif

'../data/feature_importance/12544.tif'

In [103]:
out_raster

'../model/randomforest_iterations_000/12544.tif'

In [129]:
width, height, transform = metadata_from_tile(tif)
# arma las ventanas de 100x100
windows = sliding_windows(100, 100, width, height)

In [130]:
chicho = 0
for window in list(windows):
    print(chicho)
    # origin
    src = rasterio.open(tif)
    img = src.read(window=win)
    r,m,n = img.shape
    img_df = (
                    pd.DataFrame(img.reshape(r,m*n))
                    .T.fillna(-99)
                )
    img_df.rename(columns={col:f'band_{col}' for col in img_df.columns}, inplace=True)
    # mask
    src_mask = rasterio.open(out_raster)
    mask = src_mask.read(window=win) 
    r_mask,m_mask,n_mask = mask.shape
    mask_df = (
        pd.DataFrame(mask.reshape(r_mask,m_mask*n_mask), index=['id']).T
    )
    b = mask_df.id.to_list()
    if 3 in b:
        print(mask_df)
        print(mask_df[mask_df.id==3])
    #print(mask_df.loc[mask_df.id==0])
    chicho+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [ ]:
for window, _ in tqdm(windows):
                print(window)
                win = window[0]
                
                # abre ventana en el raster original
                src = rasterio.open(tif)
                img = src.read(window=win) 
                                
                r,m,n = img.shape
                
                # arma un dataframe con la data para predecir
                # shape: 10000 filas (100x100 pixeles) x 28 columnas (bandas)
                img_df = (
                    pd.DataFrame(img.reshape(r,m*n))
                    .T.fillna(-99)
                )
                img_df.rename(columns={col:f'band_{col}' for col in img_df.columns}, inplace=True)
                
                # si no está en la primera iteración
                # arma un dataframe con la data para enmascarar
                # shape: 10000 filas (100x100 pixeles) x 28 columnas (bandas)
                if i>0:
                    # abre ventana en el raster de la iteración previa
                    # para enmascarar
                    src_mask = rasterio.open(prev_tif)
                    mask = src_mask.read(window=win) 
                    mask_df = pd.DataFrame(mask.reshape(r,m*n)).T

            

In [ ]:
while True:
   

    # separa entre nuevo train y nuevo pred
    add_to_train = predictions.query(f'pred_score >= {threshold}').copy()
    continue_pred = predictions.query(f'pred_score < {threshold}').copy()
    train_data_len, add_to_train_len , continue_pred_len = iter_train_data.shape[0], add_to_train.shape[0] , continue_pred.shape[0]
    output_pixels_file = os.path.join(output_folder, f'pixels_{n_iter}.csv')
    (
        pd.DataFrame(
            [
                [f'De entrenamiento', train_data_len],
                [f'Con proba>={threshold}', add_to_train_len],
                [f'Con proba<{threshold}', continue_pred_len]
            ],
            columns=['Pyxels_type','Pixels']
        )
        .to_csv(output_pixels_file, index=False)
    )

    output_deptos_file = os.path.join(output_folder, f'pred_deptos_{n_iter}.csv')
    (
        pd.DataFrame(
            [
                [f'De entrenamiento', train_data_len],
                [f'Con proba>={threshold}', add_to_train_len],
                [f'Con proba<{threshold}', continue_pred_len]
            ],
            columns=['Pyxels_type','Pixels']
        )
        .to_csv(output_pixels_file, index=False)
    )

    # pasa predicción a las columna id (target)
    # y lo agrega al train original
    add_to_train['id_le'] = add_to_train['pred_class']
    add_to_train['id'] = add_to_train.id_le.apply(lambda x: map_le2id.get(x))
    add_to_train['cultivo'] = add_to_train.id.apply(lambda x: map_id2cultivo.get(x))

    iter_deptos_prediction = os.path.join(output_folder,f'randomforest__deptos_prediction_{n_iter}.csv')
    (
        add_to_train
        .groupby(['cultivo','nombre'], as_index=False)
        .size()
        .rename(columns={'size':'pixels','nombre':'departamento'})
        .assign(ha=lambda x: x.pixels*0.04)
        .to_csv(iter_deptos_prediction, index=False)
    )
    iter_train_data =iter_train_data.append(add_to_train, ignore_index=True)
    iter_pred_data = continue_pred

    # imprime información
    print('''\n*** ITERACIÓN #{0:03d}
    - Modelo guardado en {1}
    - Probabilidades guardadas en {2}
    - Pixeles de entrenamiento: {3}
    - Pixeles con proba>={4}: {5}
    - Pixeles con proba<{4}: {6}'''.format(i, output_model_file, output_proba_file, train_data_len, threshold, add_to_train_len , continue_pred_len))
    i += 1
    if (add_to_train_len == 0) or (continue_pred_len == 0):
        break

# reemplaza na en columna pred_class con 'vc_original'
# (los pixeles que no tiene pred_class son los pieles de verdad de campo originales)
# y guarda la predicción final
nueva_vc_prediction = os.path.join(threshold_folder,f'randomforest_nueva_vc_prediction.csv')
iter_train_data.to_csv(nueva_vc_prediction)
nueva_vc_deptos = os.path.join(threshold_folder,f'randomforest_nueva_vc_deptos.csv')
(
    iter_train_data
    .groupby(['cultivo','nombre'], as_index=False)
    .size()
    .rename(columns={'size':'pixels','nombre':'departamento'})
    .assign(ha=lambda x: x.pixels*0.04)
    .to_csv(nueva_vc_deptos, index=False)
)


# pasa predicción a las columna id (target)
# y agrega las predicciones que no superaron el umbral a la nueva verdad de campo
# (i.e. a las que sí lo superaron)
# así, tenemos el conjunto completo y calculamos las ha sobre el total
continue_pred['id_le'] = continue_pred['pred_class']
continue_pred['id'] = continue_pred.id_le.apply(lambda x: map_le2id.get(x))
continue_pred['cultivo'] = continue_pred.id.apply(lambda x: map_id2cultivo.get(x))

total_final_prediction = os.path.join(threshold_folder,f'randomforest_total_final_prediction.csv')
total_prediction = iter_train_data.append(continue_pred)
total_prediction.to_csv(total_final_prediction)

total_final_deptos = os.path.join(threshold_folder,f'randomforest_total_final_deptos.csv')
(
    total_prediction
    .groupby(['cultivo','nombre'], as_index=False)
    .size()
    .rename(columns={'size':'pixels','nombre':'departamento'})
    .assign(ha=lambda x: x.pixels*0.04)
    .to_csv(total_final_deptos, index=False)
)